# Training NCEM to find interaction resolution

## Import

In [1]:
%load_ext autoreload
%autoreload 2

import ncem
#from ncem.data import get_data_custom, customLoader

import squidpy as sq

## Dataset

This notebook uses the same AnnData object as our interpretation notebook. 

In [2]:
ad = sq.datasets.mibitof()

We now call the NCEM trainer for the intaction model and initialize it. The data was already pre-processed, so we set the `log_transform` argument to `False`.

In [3]:
trainer = ncem.train.TrainModelInteractions()
trainer.init_estim(log_transform=False)

Next, we call the `customLoader` in ncem on the AnnData object and specify the radius we want to train the model on. 

In [4]:
trainer.estimator.data = ncem.data.customLoader(
    adata=ad, cluster='Cluster', patient='donor', library_id='library_id', radius=52
)
ncem.data.get_data_custom(interpreter=trainer.estimator)

AttributeError: module 'ncem' has no attribute 'data'

## Model

As a next step, we initialize the model. For this step, we need to specify `n_eval_nodes_per_graph`, so the number of nodes per graph that are passed through the network in each forward pass. 

In [ ]:
trainer.estimator.init_model(n_eval_nodes_per_graph=10)
trainer.estimator.model.training_model.summary()

## Training

We now train the model for 10 epochs. Usually, we would train for arround 2000 epochs to ensure the model converges. 

In [ ]:
trainer.estimator.train(epochs=10)

We trained NCEM on this dataset before in an ablation study for multiple different radii and resolution paramaters. We inspected that the model performs best at a resolution of approximately 10 $\mu m$ and that performance drops again at larger distances. 

![title](images/ablation_hartmann.png)

In general, we recommend to test a few resolution parameters to ensure convergence of NCEM and before running the type coupling analysis for the best distance. In this way, one can ensure that interactions extracted are valuable and provide vmeaningful insight into putative sender-receiver dependencies. 